In [1]:
import pandas as pd

# Load the data
minWageDf = pd.read_csv('datasets/MinimumWageData.csv', encoding='latin')

minWageDf.head()

# Drop the columns that are not needed
minWageDf = minWageDf.drop(['Footnote','Department.Of.Labor.Uncleaned.Data',
                            'Department.Of.Labor.Cleaned.Low.Value.2020.Dollars',
                            'Department.Of.Labor.Cleaned.High.Value.2020.Dollars',
                            'Department.Of.Labor.Cleaned.Low.Value',
                            'Department.Of.Labor.Cleaned.High.Value',
                            "Federal.Minimum.Wage.2020.Dollars",
                            "State.Minimum.Wage.2020.Dollars"], axis=1)


minWageDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Year                                 2862 non-null   int64  
 1   State                                2862 non-null   object 
 2   State.Minimum.Wage                   2862 non-null   float64
 3   Federal.Minimum.Wage                 2862 non-null   float64
 4   Effective.Minimum.Wage               2862 non-null   float64
 5   Effective.Minimum.Wage.2020.Dollars  2862 non-null   float64
 6   CPI.Average                          2862 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 156.6+ KB


In [2]:
# Read all the datasets
fedFundsDf = pd.read_csv('datasets/FedFunds.csv', encoding='utf-8')
gdpDf = pd.read_csv('datasets/GDP.csv', encoding='utf-8')
homePriceIndexDf = pd.read_csv('datasets/HomePriceIndex.csv', encoding='utf-8')
mortgageeDf = pd.read_csv('datasets/Mortgage.csv', encoding='utf-8')
populationGrowthDf = pd.read_csv('datasets/Population-Growth.csv', encoding='utf-8')
unemploymentDf = pd.read_csv('datasets/Unemployment-Rate.csv', encoding='utf-8')

# List of dataframes
dfs = [fedFundsDf, gdpDf, homePriceIndexDf, mortgageeDf, populationGrowthDf, unemploymentDf]

In [3]:
meanDfs = []
for df in dfs:
    if "DATE" in df.columns:
        keyCol = df.columns[1]
        df["Year"] = pd.to_datetime(df["DATE"])     # Convert the date to datetime
        df["Year"] = df["Year"].dt.year             # Extract the year
        df["Year"] = df["Year"].astype(int)         # Convert the year to integer
        dfMean = df.groupby("Year")[keyCol].mean()  # Get the mean of the main data column by year
        dfMean = dfMean.to_frame().reset_index()    # Get the Data Frame with year and mean data of the year
        meanDfs.append(dfMean)


In [4]:
# Merge the dataframes
mergedDf = minWageDf.copy()

for meanDf in meanDfs:
    mergedDf = pd.merge(mergedDf, meanDf, on="Year", how="left") # Merge the dataframes with the year as the key


In [5]:
# Cleanup column names by renaming them.
renameMap = {
    "FEDFUNDS" : "FedFundsRate",
    "CSUSHPISA" : "HomePriceIndex",
    "MORTGAGE30US" : "MortgageRate",
    "UNRATE" : "UnemploymentRate",
    "SPPOPGROWUSA" : "PopulationGrowth",
    "GDP" : "Gdp",
    "State.Minimum.Wage" : "StateMinimumWage",
    "Federal.Minimum.Wage" : "FederalMinimumWage",
    "Effective.Minimum.Wage" : "EffectiveMinimumWage",
    "CPI.Average" : "CPIAverage",
    "Effective.Minimum.Wage.2020.Dollars" : "EffectiveMinimumWage2020Dollars",
}
mergedDf = mergedDf.rename(columns=renameMap)
mergedDf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2862 entries, 0 to 2861
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             2862 non-null   int64  
 1   State                            2862 non-null   object 
 2   StateMinimumWage                 2862 non-null   float64
 3   FederalMinimumWage               2862 non-null   float64
 4   EffectiveMinimumWage             2862 non-null   float64
 5   EffectiveMinimumWage2020Dollars  2862 non-null   float64
 6   CPIAverage                       2862 non-null   float64
 7   FedFundsRate                     2862 non-null   float64
 8   Gdp                              2862 non-null   float64
 9   HomePriceIndex                   1836 non-null   float64
 10  MortgageRate                     2700 non-null   float64
 11  PopulationGrowth                 2862 non-null   float64
 12  UnemploymentRate    

In [6]:
mergedDf.sample(50)
mergedDf.to_csv("FinalDataSet.csv", index=False)